In [31]:
import os, sys, argparse, paramiko
from getpass import getpass
from os.path import exists

In [32]:
# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password:  ")

# Host information
host = "192.168.6.71"
port = 2222
username = "sarah.fornaldes"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication failed")

# TASK 2

In [33]:
sftp = ssh.open_sftp()

# Run a command to scan all known directories on Linux that have a *bin* directory
command = 'sudo -S /home/ubuntu/kraken --folder /usr/bin --folder --folder /usr/sbin/ --folder /usr/local/bin --folder /sbin --folder /usr/local/sbin --folder /bin'

ssh.exec_command('kraken')
sftp.put('kraken', 'kraken')

thePass = getpass(prompt="Please enter your SSH password:  ")

# Get the output from the command
stdin, stdout, stderr  = ssh.exec_command(command)

stdin.write(thePass + "\n")
stdin.flush()

# Get results  from stdout
lines =  stdout.readlines()
#print(lines)

# Convert the list to a string
output =  ''.join(lines)

# Header output
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'

# Footer output
sepFooter = '' + '### END ' + command + ' ###\n\n'

# Concatonate the header, output, and footer
cmd_output = sepHeader + output + sepFooter

# Save the cmd_output to a file
with open('results123.txt', 'a') as f:
    f.write(cmd_output)

#ssh.close()
#sftp.close()

In [37]:
#sftp = ssh.open_sftp()

# Uploads results.txt file
uploadFiles = ['results123.txt']

# Runs the lsof command, lsof lists all open files for a process                
command = "sudo -S lsof -i -n | awk '{print $2}' "

# Get the output from the command
stdin, stdout, stderr  = ssh.exec_command(command)

# Input password to run the command as sudo, including a new line character that provides "Enter" as if you were typing it in on a keyboard
stdin.write(thePass + "\n")

# Get results  from stdout
lines =  stdout.readlines()

# Convert the list to a string
output =  ''.join(lines)

# Header output
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'

# Footer output
sepFooter = '' + '### END ' + command + ' ###\n\n'

# Concatonate the header, output, and footer
cmd_output =  sepHeader + output +  sepFooter

# Save the cmd_output to a file
with open('resultsLsof.txt', 'a') as f:
    f.write(cmd_output)

# End SFTP Session
#ssh.close()
#sftp.close()

In [35]:
import re

resultsPID = [519423, 561559]

uploadFiles = ['resultsLsof.txt']

with open(uploadFiles) as f:

    # read in the file and save it to a variable
    contents = f.readlines()
    
    # Lists to store the results
    results = []

    # Skips first line
    for _ in range(1): 
        next(contents)

    # Loops through each line
    for eachLine in contents:
        #print(eachLine)
        # for loop searched for keywords in searchTerms;
        for keyword in contents:
            # Searches for specific keywords
            x = re.findall(r''+keyword+'', eachLine) 
            #print(x)
            for found in x:
                  results.append(found)

    # Check to see if there are results
    if len(results) == 0:
        print("No Results")
        sys.exitt(1)
        
    print(results)


TypeError: expected str, bytes or os.PathLike object, not list

In [ ]:
# https://stackoverflow.com/questions/24883009/how-to-compare-lists-whose-elements-are-regular-expressions-in-python 
list1 = ['519423', '561559']
list2 = ['593','603','603','830','855','855','935','935','561559','600020', '600170', '600390', '600489', '600523', '600626', '600716', '600818', '600854', '600947', '600952', '601052', '601067', '601169', '601242', '601357', '601383', '601485', '601534', '601633', '601653', '601752', '601759', '601861', '601881', '601977', '601982', '602084', '602108', '602204', '602220', '602318', '602330', '602429', '602470', '602568', '602604', '602705', '602900', '603028', '603036', '603134', '603284', '603390']
import re

results = ({i:j  for j in list1 for i in list2 if re.match(i,j)})
print(results)
 

{'561559': '561559'}
